In [1]:
import os
import sys
import zipfile
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append('../')
from src.data_preprocess import DataPreprocessor

pd.set_option('display.max_colwidth', 2000)

%load_ext autoreload
%autoreload 2

In [2]:
# data_folder = "//fsfshadoopstage/hadoop_stage/PEE_joint/NUS_modules/CS5228/CS5228/input"
# if len(os.listdir(data_folder))<2:
#     if not os.listdir(data_folder):
#         !kaggle competitions download -c cs5228-2022-semester-1-final-group-project -p ../input
#     Dataset = "cs5228-2022-semester-1-final-group-project"
#     with zipfile.ZipFile(f"{data_folder}/{Dataset}.zip","r") as z:
#         z.extractall("../input")

In [3]:
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        if filename.endswith('csv'):
            print(os.path.join(dirname, filename))

../input\example-submission.csv
../input\test.csv
../input\train.csv
../input\auxiliary-data\sg-commerical-centres.csv
../input\auxiliary-data\sg-mrt-stations.csv
../input\auxiliary-data\sg-primary-schools.csv
../input\auxiliary-data\sg-secondary-schools.csv
../input\auxiliary-data\sg-shopping-malls.csv
../input\auxiliary-data\sg-subzones.csv


In [4]:
train_df = pd.read_csv('../input/train.csv').reset_index()
train_df.head(2)

,index,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-street-81-adxawp85buupmsq7gwdjverc,1.414399,103.837196,0,yishun south,yishun,514500.0
1,1,259374,hdb flat for sale in 506b serangoon north avenue 4,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-estate-demrpm6ryc3l9buf846erprb,1.372597,103.875625,0,serangoon north,serangoon,995400.0


In [5]:
# print(train_df.shape)
# display(train_df.isnull().sum())
# display(train_df.info())

In [6]:
# train_df.isnull().sum()

In [7]:
# fig = px.violin(train_df, y="price", box=True, points="all", hover_data=train_df.columns)
# fig.show()

In [8]:
train_df_clean = DataPreprocessor.remove_price_outlier(train_df)
# fig = px.violin(train_df_clean, y="price", box=True, points="all", hover_data=train_df.columns)
# fig.show()

In [9]:
train_df_clean = DataPreprocessor.preprocess_title(train_df_clean)
display(train_df_clean.loc[:, ['title_property_type','property_type']].head(3))
display(train_df_clean.loc[:, ['title_n_beds','num_beds']].head(3)) ## if hdb flat,can get from property_type
display(train_df_clean.loc[:, ['title_address','address']].head(3))

,title_property_type,property_type
0,hdb flat,hdb 4 rooms
1,hdb flat,hdb
2,condo,condo


,title_n_beds,num_beds
0,hdb flat,3.0
1,hdb flat,4.0
2,4,4.0


,title_address,address
0,866 yishun street 81,sembawang / yishun (d27)
1,506b serangoon north avenue 4,hougang / punggol / sengkang (d19)
2,meyerhouse,128 meyer road


In [10]:
display(train_df_clean.loc[:, ['title_property_type','property_type', 'title_n_beds','num_beds', 'title_address','address']])

,title_property_type,property_type,title_n_beds,num_beds,title_address,address
0,hdb flat,hdb 4 rooms,hdb flat,3.0,866 yishun street 81,sembawang / yishun (d27)
1,hdb flat,hdb,hdb flat,4.0,506b serangoon north avenue 4,hougang / punggol / sengkang (d19)
2,condo,condo,4,4.0,meyerhouse,128 meyer road
3,condo,Condo,3,3.0,leedon green,26 leedon heights
4,condo,condo,2,2.0,one bernam,1 bernam street
...,...,...,...,...,...,...
20249,condo,condo,2,2.0,lentor modern,lentor central
20250,condo,Condo,2,2.0,mori,223 guillemard road
20251,condo,condo,4,4.0,pullman residences newton,18 dunearn road
20252,hdb flat,hdb 5 rooms,hdb flat,3.0,691d woodlands drive 73,admiralty / woodlands (d25)


## lat & lng

In [11]:
train_df_clean['lat'] = train_df_clean['lat'].astype(float)
train_df_clean['lng'] = train_df_clean['lng'].astype(float)
# fig = px.histogram(train_df_clean[(zscore(train_df_clean['lng']) > -1) & (zscore(train_df_clean['lng']) < 1)], x='lng')
# fig = px.histogram(train_df_clean, x='lat')
# fig.show()
# fig = px.histogram(train_df_clean, x='lng')
# fig.show()

In [12]:
# fig = px.scatter(train_df, x='lat', y='lng')
# fig.show()

In [13]:
train_df_clean = DataPreprocessor.preprocess_lat_lng(train_df_clean)

In [14]:
# fig = px.scatter(train_df_clean, x='lat', y='lng')
# fig.show()

In [15]:
def fu(sub_df):
    if((sub_df[~((sub_df['lat'] > 1) & (sub_df['lat'] < 2))].shape[0]>0) 
       & (sub_df[~((sub_df['lng'] > 103) & (sub_df['lng'] < 105))].shape[0]>0)):
        display(sub_df[["address", "title_address", 'lat', 'lng']].drop_duplicates())
train_df_clean.groupby('address').apply(fu)

""


## property_type

In [16]:
# fig = px.box(train_df_clean, x="property_type", y="price")
# fig.show()

In [17]:
train_df_clean = DataPreprocessor.preprocess_property_type(train_df_clean)

In [18]:
# fig = px.box(train_df_clean.sort_values('price'), x="property_type_clean", y="price")
# fig.update_xaxes(categoryorder='array',categoryarray = train_df_clean.groupby('property_type_clean').median().sort_values('price').index.to_list())
# fig.show()
# fig = px.scatter(train_df_clean.groupby('property_type_cat_0').median().sort_values('price').reset_index(), x="property_type_cat_0", y="price")
# fig.show()

## tenure

In [19]:
train_df['tenure'].isnull().sum()

1723

In [20]:
# fig = px.box(train_df_clean, x="tenure", y="price")
# fig.show()

In [21]:
# # train_df_clean.groupby('tenure').count()
# fig = px.histogram(train_df_clean, x='tenure')
# fig.show()

In [22]:
print(train_df['tenure'].isnull().sum())
train_df_clean = DataPreprocessor.preprocess_tenure(train_df_clean)
print(train_df_clean['tenure'].isnull().sum())

1723
0


In [23]:
# fig = px.box(train_df_clean.sort_values('price'), x="tenure", y="price")
# fig.update_xaxes(categoryorder='array',categoryarray = train_df_clean.groupby('tenure').median().sort_values('price').index.to_list())
# fig.show()
# fig = px.violin(train_df_clean,x='tenure', y="price", box=True, points="all", hover_data=['tenure_cat_0','tenure_cat_1','tenure_cat_2','tenure_cat_3'])
# fig.show()

## built_year

In [24]:
print(train_df['built_year'].isnull().sum())
train_df_clean = DataPreprocessor.preprocess_built_year(train_df_clean, uncertain=False)
print(train_df_clean['built_year'].isnull().sum())

922
567


In [25]:
# fig = px.box(train_df_clean.sort_values('price'), x="built_year", y="price", color='property_type_clean')
# fig.update_xaxes(categoryorder='array',categoryarray = train_df_clean.groupby(['built_year','property_type_clean']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

## num_beds

In [25]:
print(train_df['num_beds'].isnull().sum())
train_df_clean = DataPreprocessor.preprocess_num_beds(train_df_clean)
print(train_df_clean['num_beds'].isnull().sum())

80
0


In [30]:
# fig = px.box(train_df_clean.sort_values('price'), x="num_beds", y="price")
# fig.update_xaxes(categoryorder='array',categoryarray = train_df_clean.groupby(['num_beds']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

## num_baths

In [26]:
print(train_df['num_baths'].isnull().sum())
train_df_clean = DataPreprocessor.preprocess_num_baths(train_df_clean, False)
print(train_df_clean['num_baths'].isnull().sum())

434
1


In [32]:
# fig = px.box(train_df_clean.sort_values('price'), x="num_baths", y="price")
# fig.update_xaxes(categoryorder='array',categoryarray = train_df_clean.groupby(['num_baths']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

## size_sqft

In [27]:
print(train_df_clean['size_sqft'].isnull().sum())

0


In [54]:
# fig = px.histogram(train_df_clean, x='size_sqft')
# fig.show()

In [44]:
compare_df = {
    "size_sqft":list(train_df_clean["size_sqft"].values),
    "state":["raw_data"]*train_df_clean.shape[0]
} # removed excessively large, coverted small size unit

In [45]:
from scipy.stats import zscore
compare_df["size_sqft"]+=list(train_df[zscore(train_df['size_sqft']) < 5]['size_sqft'].values)
compare_df["state"]+=["removed excessively large"]*train_df[zscore(train_df['size_sqft']) < 5].shape[0]

In [46]:
# fig = px.histogram(train_df[zscore(train_df['size_sqft']) < 5], x='size_sqft')
# fig.show()

In [58]:
train_df_clean = DataPreprocessor.preprocess_size_sqft(train_df_clean, False)
# fig = px.histogram(train_df_clean, x='size_sqft')
# fig.show()

In [59]:
# compare_df["size_sqft"]+=list(df_['size_sqft'].values)
# compare_df["state"]+=["coverted small size unit"]*df_.shape[0]

In [60]:
# pd.DataFrame(data=compare_df)

In [61]:
# fig = px.violin(pd.DataFrame(data=compare_df), y="size_sqft", color="state", box=True, points="all")
# fig.show()

In [62]:
# fig = px.scatter(df_, x="size_sqft", y="price")
# fig.show()

## floor_level

In [63]:
print(train_df_clean['floor_level'].isnull().sum())

16638


In [84]:
train_df['floor_level'].unique(), train_df_clean['property_type_clean'].unique()

(array([nan, 'low', 'ground', 'high', 'penthouse', 'mid', 'mid (25 total)',
        'top', 'ground (9 total)', 'high (70 total)', 'low (23 total)',
        'low (17 total)', 'high (23 total)', 'high (10 total)',
        'high (25 total)', 'high (9 total)', 'high (17 total)',
        'mid (9 total)'], dtype=object),
 array(['hdb', 'condo', 'bungalow', 'terraced house', 'hdb executive',
        'apartment', 'semi-detached house', 'executive condo',
        'corner terrace', 'landed', 'cluster house', 'land only',
        'townhouse', 'conservation house', 'walk-up',
        'good class bungalow', 'shophouse'], dtype=object))

In [86]:
# fig = px.box(train_df_clean.sort_values('price'), x="floor_level", y="price")
# fig.update_xaxes(categoryorder='array',categoryarray = train_df_clean.groupby(['num_baths']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

In [87]:
train_df_clean = DataPreprocessor.preprocess_floor_level(train_df_clean)
# df_.loc[df_['floor_level_cat'].isna(), 'floor_level_cat'] = 'nan'
# df_.loc[df_['total_level_cat'].isna(), 'total_level_cat'] = 'nan'

In [85]:
# fig = px.box(df_.sort_values('price'), x="floor_level_cat", y="price", color='total_level_cat')
# fig.update_xaxes(categoryorder='array',categoryarray = df_.groupby(['floor_level_cat', 'total_level_cat']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

In [88]:
# df_['total_level_cat'].unique(), df_['floor_level_cat'].unique()

## furnishing

In [90]:
print(train_df["furnishing"].unique())

['unspecified' 'partial' 'unfurnished' 'fully' 'na']


In [91]:
df_ = DataPreprocessor.preprocess_furnishing(train_df_clean)

In [94]:
# fig = px.violin(df_,x='furnishing', y="price", box=True, points="all")
# fig.show()
fig = px.box(df_.sort_values('price'), x="furnishing", y="price", color='property_type_clean')
fig.update_xaxes(categoryorder='array',categoryarray = df_.groupby(['furnishing', 'property_type_clean']).median().sort_values('price').index.to_list())
fig.update_traces(boxpoints='outliers') 
fig.show()

## available_unit_types

In [97]:
# train_df_clean["available_unit_types"].unique()

In [95]:
df_ = DataPreprocessor.preprocess_available_unit_types(train_df_clean)

In [103]:
# fig = px.box(df_.sort_values('price'), x="number_of_types_available", y="price", color='property_type_clean')
# fig.update_xaxes(categoryorder='array',categoryarray = df_.groupby(['number_of_types_available', 'property_type_clean']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

In [106]:
fig = px.box(df_.sort_values('price'), x="max_br_available", y="price")
fig.update_xaxes(categoryorder='array',categoryarray = df_.groupby(['max_br_available']).median().sort_values('price').index.to_list())
fig.update_traces(boxpoints='outliers') 
fig.show()

In [104]:
# fig = px.box(df_.sort_values('price'), x="has_studio", y="price")
# fig.update_xaxes(categoryorder='array',categoryarray = df_.groupby(['has_studio']).median().sort_values('price').index.to_list())
# fig.update_traces(boxpoints='outliers') 
# fig.show()

## property_details_url, elevation

## planning_area

In [107]:
print(train_df_clean['planning_area'].isnull().sum())

113


In [110]:
train_df_clean[train_df_clean['planning_area'].isnull()]["property_name"].unique()

array(['1953', 'pollen & bleu', 'ness', 'm5'], dtype=object)

In [111]:
df_ = DataPreprocessor.preprocess_planning_area(train_df_clean)
print(df_['planning_area'].isnull().sum())
df_['planning_area'].unique()

0


array(['yishun', 'serangoon', 'marine parade', 'bukit timah',
       'downtown core', 'bukit batok', 'kallang', 'tanglin', 'bedok',
       'bishan', 'choa chu kang', 'novena', 'ang mo kio', 'queenstown',
       'pasir ris', 'museum', 'tampines', 'newton', 'orchard',
       'bukit panjang', 'sembawang', 'geylang', 'bukit merah', 'sengkang',
       'toa payoh', 'clementi', 'outram', 'woodlands', 'jurong west',
       'hougang', 'singapore river', 'punggol', 'river valley',
       'southern islands', 'rochor', 'jurong east', 'seletar',
       'central water catchment', 'tengah', 'lim chu kang', 'changi',
       'paya lebar', 'mandai'], dtype=object)

In [112]:
fig = px.box(df_.sort_values('price'), x="planning_area", y="price")
fig.update_xaxes(categoryorder='array',categoryarray = df_.groupby(['planning_area']).median().sort_values('price').index.to_list())
fig.update_traces(boxpoints='outliers') 
fig.show()

In [113]:
train_df = pd.read_csv('../input/train.csv')
train_df.head(2)

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-street-81-adxawp85buupmsq7gwdjverc,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north avenue 4,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-estate-demrpm6ryc3l9buf846erprb,1.372597,103.875625,0,serangoon north,serangoon,995400.0


In [114]:
sg_commercial_centres_df = pd.read_csv('../input/auxiliary-data/sg-commerical-centres.csv')
sg_commercial_centres_df.loc[sg_commercial_centres_df['type']=='IEPB', 'type'] = 'IEBP'
sg_commercial_centres_df.head(2)

,name,type,lat,lng,subzone,planning_area
0,Central Business District,CR,1.286768,103.854529,clifford pier,downtown core
1,Jurong Lake District,CR,1.334085,103.734513,lakeside (business),jurong east


In [115]:
sg_primary_schools_df = pd.read_csv('../input/auxiliary-data/sg-primary-schools.csv')
sg_primary_schools_df.head(2)

,name,lat,lng,subzone,planning_area
0,Admiralty Primary School,1.442941,103.800345,woodlands east,serangoon
1,Ahmad Ibrahim Primary School,1.433849,103.832710,yishun west,serangoon


In [116]:
sg_secondary_schools_df = pd.read_csv('../input/auxiliary-data/sg-secondary-schools.csv')
sg_secondary_schools_df.head(2)

,name,lat,lng,subzone,planning_area
0,Admiralty Secondary School,1.445912,103.802908,woodlands east,woodlands
1,Ahmad Ibrahim Secondary School,1.436095,103.830055,yishun west,yishun


In [117]:
sg_shopping_mall_df = pd.read_csv('../input/auxiliary-data/sg-shopping-malls.csv')
sg_shopping_mall_df.head(2)

,name,lat,lng,subzone,planning_area
0,10 AM,1.275568,103.863591,marina south,marina south
1,313@Somerset,1.301013,103.838540,somerset,orchard


In [118]:
train_df_clean = DataPreprocessor.data_preprocessing_v2(
    train_df, 
    test=False, 
    uncertain=False, 
    drop_na=False, 
    remove_original_attributes=True,
    if_auxiliary=True,
    sg_commercial_centres_df=sg_commercial_centres_df,
    sg_primary_schools_df=sg_primary_schools_df,
    sg_secondary_schools_df=sg_secondary_schools_df,
    sg_shopping_mall_df=sg_shopping_mall_df
)

Processed function: Function 'preprocess_available_unit_types' executed in 18.5741s & Processing function: preprocess_planning
Processed function: Function 'preprocess_secondary_school' executed in 45.6024s & Processing function: preprocess_shopping_mal


In [125]:
new_col = list(train_df_clean.columns)
new_col.remove('price')
new_col = ['price'] + new_col

df_ = train_df_clean[new_col]

In [127]:
corr = df_.corr()

In [131]:
# import plotly.io as pio
# import plotly.express as px
# import plotly.graph_objects as go

# pio.templates.default = "plotly_white"

# mask = np.triu(np.ones_like(corr, dtype=bool))
# fig = go.Heatmap(
#     z=corr.mask(mask),
#     x=corr.columns,
#     y=corr.columns,
#     colorscale=px.colors.diverging.RdBu,
#     zmin=-1,
#     zmax=1
# )
# fig.show()

In [142]:
corr

,price,built_year,num_beds,num_baths,size_sqft,lat,lng,property_type_cat_0,tenure_cat_0,tenure_cat_1,...,IHL_cat_0,pri_sch_dist,number_of_close_pri_sch,pri_sch_cat_0,sec_sch_dist,number_of_close_sec_sch,sec_sch_cat_0,shopping_mall_dist,number_of_close_shopping_mall,shopping_mall_cat_0
price,1.000000,0.058924,0.487219,0.565124,0.689098,-0.247387,-0.026531,0.585596,-0.042323,-0.301522,...,0.151328,0.321466,-0.130243,0.324314,0.340028,-0.107504,0.335748,0.185752,0.048846,0.392018
size_sqft,0.689098,-0.098475,0.710889,0.746704,1.000000,-0.099773,0.014520,0.658118,0.036821,-0.303385,...,0.026933,0.234391,-0.061199,0.232696,0.290122,-0.067063,0.239881,0.237440,-0.072486,0.291775
property_type_cat_0,0.585596,0.282329,0.470404,0.650059,0.658118,-0.303236,0.055098,1.000000,-0.097944,-0.491165,...,0.218308,0.334313,-0.257988,0.554607,0.354485,-0.160188,0.546926,0.251664,0.001925,0.583550
num_baths,0.565124,0.065034,0.849565,1.000000,0.746704,-0.090583,0.026575,0.650059,-0.006274,-0.317462,...,0.066257,0.198964,-0.094414,0.298584,0.231856,-0.076588,0.297707,0.193294,-0.041731,0.339194
floor_level_cat_6,0.509219,-0.140243,0.629400,0.659926,0.737159,-0.033542,0.035123,0.792817,0.053534,-0.381146,...,0.018817,0.149265,-0.070479,0.243230,0.207974,-0.078374,0.249941,0.253389,-0.123119,0.281694
num_beds,0.487219,-0.100885,1.000000,0.849565,0.710889,0.079465,0.015911,0.470404,0.072832,-0.217944,...,-0.064787,0.076408,0.017231,0.097344,0.123143,-0.015299,0.102571,0.171941,-0.108316,0.136505
max_br_available,0.393175,0.161302,0.382423,0.466213,0.446093,-0.178573,0.105056,0.565205,-0.384503,-0.039867,...,0.162663,0.227806,-0.161526,0.365841,0.258421,-0.109894,0.371398,0.156597,-0.044669,0.391067
shopping_mall_cat_0,0.392018,0.324229,0.136505,0.339194,0.291775,-0.563079,0.088186,0.583550,-0.160955,-0.361808,...,0.490104,0.427172,-0.296557,0.766629,0.411696,-0.175892,0.730134,0.158251,0.144135,1.000000
sec_sch_dist,0.340028,0.129947,0.123143,0.231856,0.290122,-0.360119,-0.069996,0.354485,-0.091442,-0.108077,...,0.169916,0.743161,-0.197483,0.348751,1.000000,-0.379094,0.321115,0.365845,0.049098,0.411696
sec_sch_cat_0,0.335748,0.346632,0.102571,0.297707,0.239881,-0.537938,0.074368,0.546926,-0.156201,-0.313643,...,0.454320,0.356206,-0.303879,0.773411,0.321115,-0.133504,1.000000,0.153122,0.116978,0.730134


In [147]:
import plotly.figure_factory as ff

corr = df_.corr().round(2)
corr = corr.sort_values(by='price', key=abs, ascending=False).iloc[:20]
corr = corr.loc[:,corr.index]

mask = np.triu(np.ones_like(corr, dtype=bool))
df_mask = corr.mask(mask)

fig = ff.create_annotated_heatmap(z=df_mask.to_numpy(), 
                                  x=df_mask.columns.tolist(),
                                  y=df_mask.columns.tolist(),
                                  colorscale=px.colors.diverging.RdBu,
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='Top 20 highly correlated features', 
    title_x=0.5, 
    width=1000, 
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

# NaN values are not handled automatically and are displayed in the figure
# So we need to get rid of the text manually

for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.show()